1\. Write a function that converts number representation (bin<->dec<->hex)

In [4]:
def DecimalToBinary(dec_n):
    bin_n=" " #to store result
    while(dec_n>0):
        bin_n = str(dec_n % 2) + bin_n #add dec_n % 2 at the beginning of the string
        dec_n=dec_n//2 #halve the input 
    return bin_n    


def DecimalToHexadecimal(dec_n):
    hex_n=" " #to store result
    hex_digits="0123456789ABCDEF" #hexadecimal digits
    while(dec_n>0):
        hex_n = hex_digits[dec_n % 16] + hex_n #add dec_n % 16 at the beginning of the string
        dec_n=dec_n//16 #halve the input 
    return hex_n 


def BinaryToDecimal(bin_n):
    number = 0
    counter = 0
    for i in bin_n[::-1]: # Iterating through bin_n in reverse order
        number += int(i)*(2**counter)
        counter += 1
    return number    
    

def HexadecimalToDecimal(hex_n):
    dec_n = 0
    for i,d in enumerate(hex_n) : 
        hex_digits = "0123456789ABCDEF"
        value= hex_digits.index(d) # 0 to 15
        #index = string.index(i)
        power = (len(hex_n) -(i+1)) #power of 16
        dec_n += (value*16**power)
    return dec_n


def BinaryToHexadecimal(bin_n):
    return DecimalToHexadecimal(BinaryToDecimal(bin_n))


def HexadecimalToBinary(hex_n):
    return DecimalToBinary(HexadecimalToDecimal(hex_n))
          
Decimal_number=47
Binary_number="101111"
Hexadecimal_number="2F"

print("Hexadecimal to Binary: ")
print(HexadecimalToBinary(Hexadecimal_number))

print("Hexadecimal to Decimal: ")
print(HexadecimalToDecimal(Hexadecimal_number))

print("Binary to Hexadecimal: ")
print(BinaryToHexadecimal(Binary_number))

print("Binary to Decimal: ")
print(BinaryToDecimal(Binary_number))

print("Decimal to Hexadecimal: ")
print(DecimalToHexadecimal(Decimal_number))

print("Decimal to Binary: ")
print(DecimalToBinary(Decimal_number))

Hexadecimal to Binary: 
101111 
Hexadecimal to Decimal: 
47
Binary to Hexadecimal: 
2F 
Binary to Decimal: 
47
Decimal to Hexadecimal: 
2F 
Decimal to Binary: 
101111 


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [5]:
def BinaryToDecimal(bin_n):
    number = 0
    counter = 0
    for i in bin_n[::-1]: # Iterating through bin_n in reverse order
        number += int(i)*(2**counter)
        counter += 1
    return number 

#a="123456789"
#print(a[1:6])

def ConversionToFloat(bin_float):
    if(int(bin_float[0])==0):
        sign=1
    elif(int(bin_float[0])==1):
        sign=-1    
    exponent=(BinaryToDecimal(bin_float[1:9]) - 127)
    mantissa=1 
    for i in range(0,len(bin_float[9:])):
        mantissa=mantissa+(int(bin_float[9+i])*2**(-i-1))
    return sign*mantissa*2**exponent    


print(ConversionToFloat("01101000101010111110001001010111"))
    
    

6.493599213936392e+24


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [6]:
underflow=1
overflow=1

#underflow
while(underflow!=0):
    prev_under=underflow
    underflow/=2
    
print("Underflow limit:") 
print(prev_under)


#overflow
i=0
while(i < 2000):
    try:
        overflow*=2
        i+=1
    except:
        print("Overflow:", overflow)
print("No integer overflow was found")

Underflow limit:
5e-324
No integer overflow was found


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [7]:
var1=1
var2=1

while(var1 != var1+(var2/2)):
    var2/=2

print("Machine precision: ", var2)    

Machine precision:  2.220446049250313e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [8]:
#(a)
def quadratic(a,b,c):
    x1= (-b + (b*b-4*a*c)**0.5)/(2*a)
    x2= (-b - (b*b-4*a*c)**0.5)/(2*a)
    return x1,x2

#(b)
def quadratic2(a,b,c):
    factor_m=(-b - (b*b-4*a*c))
    factor_p=(-b + (b*b-4*a*c))          
    x1= (-b + (b*b-4*a*c)**0.5)*factor_m/(2*a*factor_m)
    x2= (-b - (b*b-4*a*c)**0.5)*factor_p/(2*a*factor_p)
    return x1,x2



print(quadratic(0.001,1000, 0.001))
print(quadratic2(0.001,1000, 0.001))

(-9.999894245993346e-07, -999999.999999)
(-9.999894245993344e-07, -999999.9999989999)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [9]:
import math

delta=[1e-2, 1e-4, 1e-6, 1e-8, 1e-10, 1e-12, 1e-14]
x=1

def function(x):
    return(x*(x-1))

def derivative(x, d):
    return (function(x+d)-function(x))/d

def accuracy(der):
    return math.fabs(1-der) #1 is analitically computed derivative

for d in delta:
    print("Delta=", d)
    print("Derivative=", derivative(x,d))
    print("Accuracy=", accuracy(derivative(x,d)))
    print("\n")



Delta= 0.01
Derivative= 1.010000000000001
Accuracy= 0.010000000000000897


Delta= 0.0001
Derivative= 1.0000999999998899
Accuracy= 9.999999988985486e-05


Delta= 1e-06
Derivative= 1.0000009999177333
Accuracy= 9.99917733279787e-07


Delta= 1e-08
Derivative= 1.0000000039225287
Accuracy= 3.922528746258536e-09


Delta= 1e-10
Derivative= 1.000000082840371
Accuracy= 8.284037100736441e-08


Delta= 1e-12
Derivative= 1.0000889005833413
Accuracy= 8.890058334132256e-05


Delta= 1e-14
Derivative= 0.9992007221626509
Accuracy= 0.0007992778373491216




7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [11]:
import numpy as np
import time

def semicircle(x):
    return math.sqrt(1-x**2)

def riemann_integral(N):
    h=2/N
    integral=0
    for i in np.linspace(-1,1,N):
        integral=integral+h*semicircle(i)
    return integral    


N=100
print("N:", N)
print("Integral:",riemann_integral(N))
print("Accuratezza:",math.fabs(math.pi/2 - riemann_integral(N)))


start=time.clock()
N=10000000
integral1=riemann_integral(N)
time = time.clock()-start
print("Time:",time)

print("N:", N)
print("Integral:", riemann_integral(N))
print("Accuratezza: ", math.fabs(math.pi/2 - integral1))

N: 100
Integral: 1.5534179294048955
Accuratezza: 0.01737839739000102
Time: 54.36444100000001
N: 10000000
Integral: 1.5707961696626787
Accuratezza:  1.571322179039214e-07
